In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib
matplotlib.rcParams["figure.figsize"]=(20,10)

In [2]:
dataset=pd.read_csv("Bengaluru_House_Data.csv")
dataset.head()

,area_type,availability,location,size,society,total_sqft,bath,balcony,bhk,price
0,Super built-up Area,19-Dec,Electronic City Phase II,2 BHK,Coomee,1056,2.0,1.0,2 .0,39.07
1,Plot Area,Ready To Move,Chikka Tirupathi,4 Bedroom,Theanmp,2600,5.0,3.0,4 .0,120.00
2,Built-up Area,Ready To Move,Uttarahalli,3 BHK,NaN,1440,2.0,3.0,3,62.00
3,Super built-up Area,Ready To Move,Lingadheeranahalli,3 BHK,Soiewre,1521,3.0,1.0,3 .0,95.00
4,Super built-up Area,Ready To Move,Kothanur,2 BHK,NaN,1200,2.0,1.0,2 .0,51.00


In [3]:
dataset.groupby('area_type')['area_type'].agg('count')

area_type
Built-up  Area          2418
Carpet  Area              87
Plot  Area              2025
Super built-up  Area    8789
Super built-up Area        1
Name: area_type, dtype: int64

In [4]:
dataset1=dataset.drop(['area_type','society','availability','balcony','bhk'],axis='columns')
dataset1.head()

,location,size,total_sqft,bath,price
0,Electronic City Phase II,2 BHK,1056,2.0,39.07
1,Chikka Tirupathi,4 Bedroom,2600,5.0,120.00
2,Uttarahalli,3 BHK,1440,2.0,62.00
3,Lingadheeranahalli,3 BHK,1521,3.0,95.00
4,Kothanur,2 BHK,1200,2.0,51.00


In [5]:
dataset1.isnull().sum()

location       1
size          16
total_sqft     0
bath          73
price          0
dtype: int64

In [6]:
d3=dataset1.dropna()
d3.head()

,location,size,total_sqft,bath,price
0,Electronic City Phase II,2 BHK,1056,2.0,39.07
1,Chikka Tirupathi,4 Bedroom,2600,5.0,120.00
2,Uttarahalli,3 BHK,1440,2.0,62.00
3,Lingadheeranahalli,3 BHK,1521,3.0,95.00
4,Kothanur,2 BHK,1200,2.0,51.00


In [7]:
d3['size'].unique()
#clean this as it contains diff values like BHK,bedroom ; we,ll use tokenization to take only the starting value

array(['2 BHK', '4 Bedroom', '3 BHK', '4 BHK', '6 Bedroom', '3 Bedroom',
       '1 BHK', '1 RK', '1 Bedroom', '8 Bedroom', '2 Bedroom',
       '7 Bedroom', '5 BHK', '7 BHK', '6 BHK', '5 Bedroom', '11 BHK',
       '9 BHK', '9 Bedroom', '27 BHK', '10 Bedroom', '11 Bedroom',
       '10 BHK', '19 BHK', '16 BHK', '43 Bedroom', '14 BHK', '8 BHK',
       '12 Bedroom', '13 BHK', '18 Bedroom'], dtype=object)

In [8]:
d3['bhk']=d3['size'].apply(lambda x: int(x.split()[0]))

/var/folders/g8/8k617n7557z446m852rmncfw0000gn/T/ipykernel_23481/2961131244.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d3['bhk']=d3['size'].apply(lambda x: int(x.split()[0]))


In [9]:
d3.head()

,location,size,total_sqft,bath,price,bhk
0,Electronic City Phase II,2 BHK,1056,2.0,39.07,2
1,Chikka Tirupathi,4 Bedroom,2600,5.0,120.00,4
2,Uttarahalli,3 BHK,1440,2.0,62.00,3
3,Lingadheeranahalli,3 BHK,1521,3.0,95.00,3
4,Kothanur,2 BHK,1200,2.0,51.00,2


In [10]:
d3.total_sqft.unique()
#we'll clean the 1133-1384 type values by average of the 2 and remove all other with sq.ft or metre 
#we'll find all the values with '1133-1384' such type of values 

array(['1056', '2600', '1440', ..., '1133-1384', '774', '4689'],
      dtype=object)

In [11]:
def is_float(x):
    try:
        float(x)
    except:
        return False
    return True 


In [12]:
d3[~d3['total_sqft'].apply(is_float)].head(10)

,location,size,total_sqft,bath,price,bhk
30,Yelahanka,4 BHK,2100-2850,4.0,186.000,4
122,Hebbal,4 BHK,3067-8156,4.0,477.000,4
137,8th Phase JP Nagar,2 BHK,1042-1105,2.0,54.005,2
165,Sarjapur,2 BHK,1145-1340,2.0,43.490,2
188,KR Puram,2 BHK,1015-1540,2.0,56.800,2
410,Kengeri,1 BHK,34.46Sq. Meter,1.0,18.500,1
549,Hennur Road,2 BHK,1195-1440,2.0,63.770,2
648,Arekere,9 Bedroom,4125Perch,9.0,265.000,9
661,Yelahanka,2 BHK,1120-1145,2.0,48.130,2
672,Bettahalsoor,4 Bedroom,3090-5002,4.0,445.000,4


In [13]:
#remove all the diff values or u may convert the Sq.Meter to total_sqft

def convert_sqft_to_num(x):
    tokens=x.split('-')
    if len(tokens)==2:
        return (float(tokens[0])+float(tokens[1]))/2
    try:
        return float(x)
    except:
        None
     

In [14]:
d4=d3.copy()
d4['total_sqft']=d4['total_sqft'].apply(convert_sqft_to_num)
d4.head()

,location,size,total_sqft,bath,price,bhk
0,Electronic City Phase II,2 BHK,1056.0,2.0,39.07,2
1,Chikka Tirupathi,4 Bedroom,2600.0,5.0,120.00,4
2,Uttarahalli,3 BHK,1440.0,2.0,62.00,3
3,Lingadheeranahalli,3 BHK,1521.0,3.0,95.00,3
4,Kothanur,2 BHK,1200.0,2.0,51.00,2


In [15]:
d4['price_per_sqft']=d4['price']*100000/d4['total_sqft']   #in lacs so multipyling by `100000`
d4.head()

,location,size,total_sqft,bath,price,bhk,price_per_sqft
0,Electronic City Phase II,2 BHK,1056.0,2.0,39.07,2,3699.810606
1,Chikka Tirupathi,4 Bedroom,2600.0,5.0,120.00,4,4615.384615
2,Uttarahalli,3 BHK,1440.0,2.0,62.00,3,4305.555556
3,Lingadheeranahalli,3 BHK,1521.0,3.0,95.00,3,6245.890861
4,Kothanur,2 BHK,1200.0,2.0,51.00,2,4250.000000


In [16]:
#feature engineering 

len(d4.location.unique())

#we'll reduce the locations of less than 10 properties to other locations otherwise it will give 1304 columns of data 

1304

In [17]:
d4.location=d4.location.apply(lambda x: x.strip())
location_stats=d4.groupby('location')['location'].agg('count').sort_values(ascending=True)
location_stats

location
1 Annasandrapalya      1
Kudlu Village,         1
Kumbhena Agrahara      1
Kuvempu Layout         1
LIC Colony             1
                    ... 
Thanisandra          236
Kanakpura Road       266
Electronic City      304
Sarjapur  Road       392
Whitefield           535
Name: location, Length: 1293, dtype: int64

In [18]:
len(location_stats[location_stats<=10])
#we'll categorise this data into other location category

1052

In [19]:
location_stats10=location_stats[location_stats<=10]
location_stats10

location
1 Annasandrapalya       1
Kudlu Village,          1
Kumbhena Agrahara       1
Kuvempu Layout          1
LIC Colony              1
                       ..
Kalkere                10
Naganathapura          10
Sector 1 HSR Layout    10
Basapura               10
BTM 1st Stage          10
Name: location, Length: 1052, dtype: int64

In [20]:
len(d4.location.unique())

1293

In [21]:

d4.location = d4.location.apply(lambda x: 'other' if x in location_stats10 else x)
len(d4.location.unique())

242

In [22]:
#outlier(data errors) detection and removal 
#i.e any 2BHK appartnment will not have total_sqft less than 500 in number i.e typically 1BHK for 300

d4[d4.total_sqft/d4.bhk<500].head()

,location,size,total_sqft,bath,price,bhk,price_per_sqft
2,Uttarahalli,3 BHK,1440.0,2.0,62.00,3,4305.555556
8,Marathahalli,3 BHK,1310.0,3.0,63.25,3,4828.244275
9,other,6 Bedroom,1020.0,6.0,370.00,6,36274.509804
16,Bisuvanahalli,3 BHK,1180.0,3.0,48.00,3,4067.796610
25,other,3 BHK,1250.0,3.0,56.00,3,4480.000000


In [23]:
d5=d4[~(d4.total_sqft/d4.bhk<500)]   #removed the above rows [~=negate]
d5.shape

(9201, 7)

In [24]:
#we'll look for outliner in price per sqft 

d5.price_per_sqft.describe()

count     9155.000000
mean      6316.594050
std       3720.088045
min        267.829813
25%       4320.000000
50%       5462.559242
75%       7028.930066
max      76923.076923
Name: price_per_sqft, dtype: float64

In [25]:
#min=267 which is very low rate and max is very high for a property to be so we'll remove them based on standard deviation 
#that is we'll remove anything which is beyond one standard deviation 

def remove_outliers(df):
    df_out =pd.DataFrame()
    for key,subdf in df.groupby('location'):
        m=np.mean(subdf.price_per_sqft)
        st=np.std(subdf.price_per_sqft)
        reduced_df=subdf[(subdf.price_per_sqft>(m-st))& (subdf.price_per_sqft<=(m+st))]
        df_out=pd.concat([df_out,reduced_df],ignore_index=True)
    return df_out

d6=remove_outliers(d5)
d6.shape    

(7205, 7)

In [26]:
#we'll check if property rate for 2BHK is more than 3BHK 
#function to remove the above outliner 

def remove_bhk_outliers(df):
    exclude_indices = np.array([])
    for location, location_df in df.groupby( 'location'):
        bhk_stats = {}
        for bhk, bhk_df in location_df. groupby ('bhk'):
            bhk_stats [bhk] = {
            'mean' : np.mean(bhk_df.price_per_sqft),
            'std': np. std (bhk_df.price_per_sqft),
            'count': bhk_df. shape[0]
            }
        for bhk, bhk_df in location_df.groupby('bhk'): 
            stats = bhk_stats. get (bhk-1)
            if stats and stats[ 'count']>5:
                exclude_indices = np. append (exclude_indices, bhk_df[bhk_df.price_per_sqft<(stats['mean'])]. index.values)
    return df.drop(exclude_indices,axis='index')

d7=remove_bhk_outliers(d6)
d7.shape

(5447, 7)

In [27]:
#if no. of bathrooms are greater than the bedroom+2, we'll mark them as outlier and remove them

d7[d7.bath>d7.bhk+2]

,location,size,total_sqft,bath,price,bhk,price_per_sqft
1070,Chikkabanavar,4 Bedroom,2460.0,7.0,80.0,4,3252.032520
4698,Thanisandra,3 BHK,1806.0,6.0,116.0,3,6423.034330
5948,other,6 BHK,11338.0,9.0,1000.0,6,8819.897689


In [28]:
d8=d7[d7.bath<d7.bhk+2]
d8.shape

(5380, 7)

In [29]:
#for model we do not need size and price_per_sqft as we created them to find the outlier, so we'll remove them 

d9=d8.drop(['size','price_per_sqft'],axis='columns')
d9.head()

,location,total_sqft,bath,price,bhk
0,1st Block Jayanagar,2850.0,4.0,428.0,4
1,1st Block Jayanagar,1630.0,3.0,194.0,3
2,1st Block Jayanagar,1875.0,2.0,235.0,3
3,1st Block Jayanagar,1235.0,2.0,148.0,2
4,1st Block Jayanagar,2750.0,4.0,413.0,4


In [30]:
#one hot encoder will be used to convert the location column to binary number to be understood by the machine 
#we'll do this by dummy encoding 

dummies = pd.get_dummies(d9.location).astype(int)
dummies.head()

,1st Block Jayanagar,1st Phase JP Nagar,2nd Phase Judicial Layout,2nd Stage Nagarbhavi,5th Block Hbr Layout,5th Phase JP Nagar,6th Phase JP Nagar,7th Phase JP Nagar,8th Phase JP Nagar,9th Phase JP Nagar,...,Vishveshwarya Layout,Vishwapriya Layout,Vittasandra,Whitefield,Yelachenahalli,Yelahanka,Yelahanka New Town,Yelenahalli,Yeshwanthpur,other
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [31]:
d10=pd.concat([d9,dummies.drop('other',axis='columns')],axis='columns')
d10.head()

,location,total_sqft,bath,price,bhk,1st Block Jayanagar,1st Phase JP Nagar,2nd Phase Judicial Layout,2nd Stage Nagarbhavi,5th Block Hbr Layout,...,Vijayanagar,Vishveshwarya Layout,Vishwapriya Layout,Vittasandra,Whitefield,Yelachenahalli,Yelahanka,Yelahanka New Town,Yelenahalli,Yeshwanthpur
0,1st Block Jayanagar,2850.0,4.0,428.0,4,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1st Block Jayanagar,1630.0,3.0,194.0,3,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1st Block Jayanagar,1875.0,2.0,235.0,3,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1st Block Jayanagar,1235.0,2.0,148.0,2,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1st Block Jayanagar,2750.0,4.0,413.0,4,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [32]:
d11=d10.drop('location',axis='columns')
d11.head()

,total_sqft,bath,price,bhk,1st Block Jayanagar,1st Phase JP Nagar,2nd Phase Judicial Layout,2nd Stage Nagarbhavi,5th Block Hbr Layout,5th Phase JP Nagar,...,Vijayanagar,Vishveshwarya Layout,Vishwapriya Layout,Vittasandra,Whitefield,Yelachenahalli,Yelahanka,Yelahanka New Town,Yelenahalli,Yeshwanthpur
0,2850.0,4.0,428.0,4,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1630.0,3.0,194.0,3,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1875.0,2.0,235.0,3,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1235.0,2.0,148.0,2,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2750.0,4.0,413.0,4,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [33]:
x=d11.drop('price',axis='columns')   #all columns except the price one 
y=d11.price                          #the price column

In [34]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=10)

In [35]:
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(x_train, y_train)
regressor.score(x_test,y_test)

0.8498296496792423

In [36]:
#we'll use gridsearrchCV 
#it will try different different regression models and will give the best one

# from sklearn.model_selection import GridSearchCV
# from sklearn.linear_model import LinearRegression, Lasso
# from sklearn.tree import DecisionTreeRegressor
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.model_selection import ShuffleSplit
# import pandas as pd

# models = {
#     'Linear Regression': (LinearRegression(), {'fit_intercept': [True]}),
#     'Lasso': (Lasso(), {'alpha': [0.1, 1, 10], 'selection': ['random', 'cyclic']}),
#     'Decision Tree': (DecisionTreeRegressor(), {'criterion': ['friedman_mse', 'poisson', 'squared_error', 'absolute_error'], 'splitter': ['best', 'random']}),
#     'Random Forest': (RandomForestRegressor(), {'n_estimators': [50, 100, 200], 'max_depth': [None, 10, 20]})
# }

# cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
# results = []

# for model_name, (model, param_grid) in models.items():
#     grid_search = GridSearchCV(model, param_grid, cv=cv, scoring='r2')
#     grid_search.fit(x, y)
    
#     results.append({
#         'Model': model_name,
#         'Best Parameters': grid_search.best_params_,
#         'Best R-squared Score': grid_search.best_score_
#     })

# results_df = pd.DataFrame(results)
# print(results_df)


In [37]:
x.columns

Index(['total_sqft', 'bath', 'bhk', '1st Block Jayanagar',
       '1st Phase JP Nagar', '2nd Phase Judicial Layout',
       '2nd Stage Nagarbhavi', '5th Block Hbr Layout', '5th Phase JP Nagar',
       '6th Phase JP Nagar',
       ...
       'Vijayanagar', 'Vishveshwarya Layout', 'Vishwapriya Layout',
       'Vittasandra', 'Whitefield', 'Yelachenahalli', 'Yelahanka',
       'Yelahanka New Town', 'Yelenahalli', 'Yeshwanthpur'],
      dtype='object', length=240)

In [38]:
#location index will work like this 
#x.columns=='1st Block Jayanagar' this gives out a tuple [0] helps to find the first array of the tuple and [0] helps to find the first element of that array
location_index=np.where(x.columns=='1st Block Jayanagar')[0][0]
location_index    

3

In [39]:
#now we'll create a function to predict the values

def predict_price(location,sqft,bath,bhk):
    location_index=np.where(x.columns==location)[0][0]
    features=np.zeros(len(x.columns))
    features[0]=sqft
    features[1]=bath
    features[2]=bhk
    if location_index>=0:
        features[location_index]=1
    return regressor.predict([features])[0]    
    

In [40]:
predict_price('1st Phase JP Nagar',1000,2,3)

/Users/parthbehl/micromamba/lib/python3.9/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


79.30281361021046

In [41]:
predict_price('1st Phase JP Nagar',1000,3,3)

/Users/parthbehl/micromamba/lib/python3.9/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


81.62684878561578

In [42]:
#now we'll deploy the model on the python flask server 
#for that we'll create a pickle file for the model 

import pickle 
with open('house_prediction_prices.pickle','wb') as f:
    pickle.dump(regressor,f)

In [43]:
#json file to store all the columns of our data
#we'll convert the all columns values to lower to have easy access
import json
columns={
    'data_columns':[col.lower() for col in x.columns]
}
with open('columns.json','w') as f:
    f.write(json.dumps(columns))